In [1]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import GRU, LSTM
from keras.layers import Conv1D, MaxPooling1D
import numpy as np
import pandas as pd
import random
from sklearn import tree
import copy
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.utils import resample
from keras.layers import Dropout, Dense, GRU, Embedding
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout, Dense,Input,Embedding,Flatten, AveragePooling2D, Conv2D,Reshape
from keras.models import Sequential,Model

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
pro_file = './dataset-baselines/all-policy/pro.txt'
anti_file = './dataset-baselines/all-policy/anti.txt'
neutral_file = './dataset-baselines/all-policy/neutral.txt'

In [3]:
def read_into_string(filename):
    text_file = open(filename, 'r')
    lines = text_file.read().split('\n')
    text_file.close()
    return lines

In [4]:
pro = read_into_string(pro_file) #1
anti = read_into_string(anti_file) #0
neutral = read_into_string(neutral_file) #2

In [5]:
len(pro), len(anti), len(neutral)

(943, 352, 772)

In [6]:
def split(dataset, label, train_test_split = 0.85):
    random.shuffle(dataset)
    idx = int(len(dataset) * train_test_split)
    train = dataset[: idx]
    test = dataset[idx + 1:]
    y_train = [label for x in range(len(train))]
    y_test = [label for x in range(len(test))]
    return (train, y_train), (test, y_test)

In [7]:
def balance_classes(X_train, y_train):
    dic = {}
    minm = 10000000
    for x, y in zip(X_train, y_train):
        if y in dic.keys():
            dic[y] = (1 + dic[y][0], [x] + dic[y][1])
        else:
            dic[y] = (1, [x])
    for k in dic.keys():
        if dic[k][0] < minm:
            minm = dic[k][0]
    X_train = []
    y_train = []
    for k in dic.keys():
        X_train += dic[k][1][: minm]
        y_train += [k for x in range(minm)]
    return shuffle(X_train, y_train)

In [8]:
def upsample(X_train, y_train):
    
    dic = {}
    minm = 10000000
    maxm = -minm
    X_final = []
    y_final = []
    
    # create the dictionary
    for x, y in zip(X_train, y_train):
        if y in dic.keys():
            dic[y] = (1 + dic[y][0], [x] + dic[y][1])
        else:
            dic[y] = (1, [x])
    
    # find maximum and minimum
    for k in dic.keys():
        count = dic[k][0]
        if dic[k][0] < minm:
            minm = count
        if dic[k][0] > maxm:
            maxm = count
            
    # Upsample all the non-majority classes
    for k in dic.keys():
        count = dic[k][0]
        examples = dic[k][1]
        if count < maxm:
            examples = resample(examples, 
                                          replace=True, # sample with replacement
                                          n_samples=maxm, # match number in majority class
                                          random_state=27) # reproducible results
            assert len(examples) == maxm
        X_final += examples
        y_final += [k for x in range(len(examples))]
        
    return shuffle(X_final, y_final)

In [10]:
(pro_X_train, pro_y_train), (pro_X_test, pro_y_test) = split(copy.deepcopy(pro), 1)
(anti_X_train, anti_y_train), (anti_X_test, anti_y_test) = split(copy.deepcopy(anti), 0)
(neutral_X_train, neutral_y_train), (neutral_X_test, neutral_y_test) = split(copy.deepcopy(neutral), 2)

In [10]:
# pro v/s anti classification
# pro:1, anti:0
# X_train = pro_X_train + anti_X_train
# y_train = pro_y_train + anti_y_train
# X_test = pro_X_test + anti_X_test
# y_test = pro_y_test + anti_y_test
# X_train, y_train = shuffle(X_train, y_train)
# X_test, y_test = shuffle(X_test, y_test)

In [11]:
# # pro v/s anti v/s neutral classification
# pro:1, anti:0, neutral:2
# X_train = pro_X_train + anti_X_train + neutral_X_train
# y_train = pro_y_train + anti_y_train + neutral_y_train
# X_test = pro_X_test + anti_X_test + neutral_X_test
# y_test = pro_y_test + anti_y_test + neutral_y_test
# X_train, y_train = shuffle(X_train, y_train)
# X_test, y_test = shuffle(X_test, y_test)

In [12]:
# neutral v/s non-neutral classification
# neutral:2, non-neutral:1
X_train = pro_X_train + anti_X_train + neutral_X_train
y_train = [1 for x in range(len(pro_y_train + anti_y_train))] + neutral_y_train
X_test = pro_X_test + anti_X_test + neutral_X_test
y_test = [1 for x in range(len(pro_y_test + anti_y_test))] + neutral_y_test
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [13]:
undersample = True

In [14]:
if undersample:
    X_train, y_train = balance_classes(X_train, y_train)
else:
    X_train, y_train = upsample(X_train, y_train)

### SVM

In [ ]:
def svm(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
svm(X_train, y_train, X_test, y_test)

### Rochhio Classification

In [ ]:
def rocchio(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', NearestCentroid()),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
rocchio(X_train, y_train, X_test, y_test)

### Random Forest Classifier

In [ ]:
def random_forest(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', RandomForestClassifier(n_estimators=100)),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

### RNN

In [ ]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/Users/ankursharma/Desktop/MTP-org/dataset/glove.6B.50d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [ ]:
def Build_Model_RNN_Text(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=50, dropout=0.5):
    """
    def buildModel_RNN(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
    word_index in word index ,
    embeddings_index is embeddings index, look at data_helper.py
    nClasses is number of classes,
    MAX_SEQUENCE_LENGTH is maximum lenght of text sequences
    """

    model = Sequential()
    hidden_layer = 3
    gru_node = 256

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) != len(embedding_vector):
                print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                      "into shape", str(len(embedding_vector)), " Please make sure your"
                                                                " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))


    print(gru_node)
    for i in range(0,hidden_layer):
        model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))
        model.add(Dropout(dropout))
    model.add(GRU(gru_node, recurrent_dropout=0.2))
   #model.add(Dense(, activation='relu'))
    model.add(Dense(nclasses, activation='softmax'))


    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model


In [ ]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)


model_RNN = Build_Model_RNN_Text(word_index,embeddings_index, len(np.unique(X_test)))

model_RNN.summary()

model_RNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=20,
                              batch_size=64,
                              verbose=2)

predicted = model_RNN.predict_classes(X_test_Glove)

print(metrics.classification_report(y_test, predicted))

### RCNN

In [ ]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/Users/ankursharma/Desktop/MTP-org/dataset/glove.6B.50d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [ ]:
def Build_Model_RCNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=50):

    kernel_size = 2
    filters = 256
    pool_size = 2
    gru_node = 256

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector



    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)


model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, len(np.unique(y_test)))


model_RCNN.summary()

model_RCNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=200,
                              batch_size=32,
                              verbose=2)

predicted = model_RCNN.predict(X_test_Glove)

predicted = np.argmax(predicted, axis=1)
print(metrics.classification_report(y_test, predicted))

In [ ]:
model_RCNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=200,
                              batch_size=32,
                              verbose=2)

predicted = model_RCNN.predict(X_test_Glove)

predicted = np.argmax(predicted, axis=1)
print(metrics.classification_report(y_test, predicted))

### Multinomial Naive Bayes

In [ ]:
def naive_bayes(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
naive_bayes(X_train, y_train, X_test, y_test)

### K Nearest Neighbors

In [ ]:
def knn(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', KNeighborsClassifier()),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
knn(X_train, y_train, X_test, y_test)

### Decision Trees

In [ ]:
def decision_trees(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', tree.DecisionTreeClassifier()),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
decision_trees(X_train, y_train, X_test, y_test)

### Deep Neural Networks

In [16]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

In [17]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [18]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)


model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], len(np.unique(X_test)))
model_DNN.summary()
exit(1)
model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              epochs=200,
                              batch_size=32,
                              verbose=2)

predicted = model_DNN.predict_classes(X_test_tfidf)

print(metrics.classification_report(y_test, predicted))

tf-idf with 3265 features
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1672192   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)             

Epoch 66/200
 - 1s - loss: 0.0054 - accuracy: 0.9983 - val_loss: 2.7789 - val_accuracy: 0.6943
Epoch 67/200
 - 1s - loss: 0.0053 - accuracy: 0.9967 - val_loss: 2.8302 - val_accuracy: 0.6995
Epoch 68/200
 - 1s - loss: 0.0033 - accuracy: 0.9983 - val_loss: 2.6670 - val_accuracy: 0.6995
Epoch 69/200
 - 1s - loss: 0.0040 - accuracy: 0.9967 - val_loss: 2.6823 - val_accuracy: 0.6943
Epoch 70/200
 - 1s - loss: 0.0022 - accuracy: 1.0000 - val_loss: 2.8389 - val_accuracy: 0.6943
Epoch 71/200
 - 1s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 3.0023 - val_accuracy: 0.6943
Epoch 72/200
 - 1s - loss: 0.0048 - accuracy: 0.9983 - val_loss: 2.9959 - val_accuracy: 0.7202
Epoch 73/200
 - 1s - loss: 0.0026 - accuracy: 0.9983 - val_loss: 2.8598 - val_accuracy: 0.7254
Epoch 74/200
 - 1s - loss: 0.0077 - accuracy: 0.9983 - val_loss: 3.1607 - val_accuracy: 0.6839
Epoch 75/200
 - 1s - loss: 0.0045 - accuracy: 0.9967 - val_loss: 3.0440 - val_accuracy: 0.6891
Epoch 76/200
 - 1s - loss: 0.0013 - accuracy: 1.00

Epoch 152/200
 - 1s - loss: 0.0023 - accuracy: 0.9983 - val_loss: 8.0151 - val_accuracy: 0.6114
Epoch 153/200
 - 1s - loss: 0.0082 - accuracy: 0.9983 - val_loss: 6.0526 - val_accuracy: 0.6425
Epoch 154/200
 - 1s - loss: 0.0033 - accuracy: 0.9983 - val_loss: 4.4865 - val_accuracy: 0.7047
Epoch 155/200
 - 1s - loss: 0.0165 - accuracy: 0.9950 - val_loss: 7.7092 - val_accuracy: 0.6010
Epoch 156/200
 - 1s - loss: 0.0074 - accuracy: 0.9967 - val_loss: 4.3285 - val_accuracy: 0.6943
Epoch 157/200
 - 1s - loss: 0.0104 - accuracy: 0.9950 - val_loss: 3.5906 - val_accuracy: 0.7461
Epoch 158/200
 - 1s - loss: 0.0037 - accuracy: 0.9967 - val_loss: 3.6867 - val_accuracy: 0.7513
Epoch 159/200
 - 1s - loss: 0.0016 - accuracy: 1.0000 - val_loss: 3.8695 - val_accuracy: 0.7513
Epoch 160/200
 - 1s - loss: 0.0034 - accuracy: 0.9967 - val_loss: 3.8333 - val_accuracy: 0.7513
Epoch 161/200
 - 1s - loss: 0.0027 - accuracy: 0.9983 - val_loss: 4.8455 - val_accuracy: 0.7254
Epoch 162/200
 - 1s - loss: 0.0019 - acc

### CNN

In [ ]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/Users/ankursharma/Desktop/MTP-org/dataset/glove.6B.100d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [ ]:
def Build_Model_CNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=100, dropout=0.5):

    """
        def buildModel_CNN(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=100, dropout=0.5):
        word_index in word index ,
        embeddings_index is embeddings index, look at data_helper.py
        nClasses is number of classes,
        MAX_SEQUENCE_LENGTH is maximum lenght of text sequences,
        EMBEDDING_DIM is an int value for dimention of word embedding look at data_helper.py
    """

    model = Sequential()
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)

    # applying a more complex convolutional approach
    convs = []
    filter_sizes = []
    layer = 5
    print("Filter  ",layer)
    for fl in range(0,layer):
        filter_sizes.append((fl+2,fl+2))

    node = 128
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    emb = Reshape((MAX_SEQUENCE_LENGTH,10, 10), input_shape=(MAX_SEQUENCE_LENGTH,100))(embedded_sequences)

    for fsz in filter_sizes:
        l_conv = Conv2D(node, padding="same", kernel_size=fsz, activation='relu')(emb)
        l_pool = AveragePooling2D(pool_size=(5,1), padding="same")(l_conv)
        #l_pool = Dropout(0.25)(l_pool)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)
    l_cov1 = Conv2D(node, (5,5), padding="same", activation='relu')(l_merge)
    l_cov1 = AveragePooling2D(pool_size=(5,2), padding="same")(l_cov1)
    l_cov2 = Conv2D(node, (5,5), padding="same", activation='relu')(l_cov1)
    l_pool2 = AveragePooling2D(pool_size=(5,2), padding="same")(l_cov2)
    l_cov2 = Dropout(dropout)(l_pool2)
    l_flat = Flatten()(l_cov2)
    l_dense = Dense(128, activation='relu')(l_flat)
    l_dense = Dropout(dropout)(l_dense)

    preds = Dense(nclasses, activation='softmax')(l_dense)
    model = Model(sequence_input, preds)

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])



    return model

In [ ]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)


model_CNN = Build_Model_CNN_Text(word_index,embeddings_index, len(np.unique(X_test)))


model_CNN.summary()

model_CNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=100,
                              batch_size=32,
                              verbose=2)

predicted = model_CNN.predict(X_test_Glove)

predicted = np.argmax(predicted, axis=1)


print(metrics.classification_report(y_test, predicted))

### Gradient Boosting Classifier

In [ ]:
def gradient_boosting(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', GradientBoostingClassifier(n_estimators=50,verbose=2)),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
gradient_boosting(X_train, y_train, X_test, y_test)

### Bag of Words

In [ ]:
def bow(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', BaggingClassifier(KNeighborsClassifier())),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
bow(X_train, y_train, X_test, y_test)